In [4]:
import random
from classes.minion import Minion
from classes.hero import Hero
from scenarios.deck_1 import deck  # Example deck import
from solver.slv import run_single_turn
from state.transition_state import start_turn, apply_results, end_turn

In [5]:
from classes.minion import Minion

# Friendly Board (4 minions)
friendly_minions = [
    Minion(
        name="FriendlyMinion A",
        minion_class="Neutral",
        keywords=[],
        attack=2,
        health=2,
        strat_value=1,
        mana_cost=2
    ),
    Minion(
        name="FriendlyMinion B",
        minion_class="Neutral",
        keywords=[],
        attack=2,
        health=3,
        strat_value=1,
        mana_cost=3
    ),
    Minion(
        name="FriendlyMinion C",
        minion_class="Neutral",
        keywords=[],
        attack=3,
        health=2,
        strat_value=1,
        mana_cost=2
    ),
    Minion(
        name="FriendlyMinion D",
        minion_class="Neutral",
        keywords=[],
        attack=1,
        health=1,
        strat_value=1,
        mana_cost=1
    )
]

# Enemy Board (3 minions)
enemy_minions = [
    Minion(
        name="EnemyMinion #1",
        minion_class="Neutral",
        keywords=[],
        attack=2,
        health=2,
        strat_value=1,
        mana_cost=1
    ),
    Minion(
        name="EnemyMinion #2",
        minion_class="Neutral",
        keywords=["Taunt", "Divine Shield"],
        attack=3,
        health=3,
        strat_value=1,
        mana_cost=3
    ),
    Minion(
        name="EnemyMinion #3",
        minion_class="Neutral",
        keywords=[],
        attack=1,
        health=1,
        strat_value=1,
        mana_cost=1
    )
]

# Hand is empty
hand_list = []

In [6]:
def main():
    # 1) Create Hero objects (assuming a Hero class exists)
    hero1 = Hero("Paladin", 30)
    hero2 = Hero("Warrior", 30)

    # 2) Print the deck size before drawing
    print(f"Deck has {len(deck)} cards at the start.")

    # 3) Start turn: hero1 draws exactly one card from the deck
    start_turn(hero1, deck, hand_list)

    # Check what got drawn
    print(f"After draw, hand_list has {len(hand_list)} cards.")
    for k, card in enumerate(hand_list):
        print(f"  Hand card {k}: {card.name} "
              f"(Atk={card.attack}, HP={card.health}, Cost={card.mana_cost})")

    # 4) Build arrays for the solver
    m = len(friendly_minions)   # number of friendly minions on board
    n = len(enemy_minions)      # number of enemy minions on board
    h = len(hand_list)          # number of cards in hand
    M = 5                       # Mana for this turn
    H_hero = 16                 # Opponent hero health

    print(f"Number of friendly minions on board: {m}")
    for i, fmn in enumerate(friendly_minions):
        print(f"  - Friendly Minion {i}: {fmn.name} "
              f"(Atk={fmn.attack}, HP={fmn.health}, Keywords={fmn.keywords})")

    print(f"Number of enemy minions on board: {n}")
    for j, emn in enumerate(enemy_minions):
        print(f"  - Enemy Minion {j}: {emn.name} "
              f"(Atk={emn.attack}, HP={emn.health}, Keywords={emn.keywords})")

    print(f"Number of cards in hand: {h}")
    for k, card in enumerate(hand_list):
        print(f"  - Hand Card {k}: {card.name} "
              f"(Atk={card.attack}, HP={card.health}, Cost={card.mana_cost}, Keywords={card.keywords})")

    # Combine board minions + hand minions for indexing in the solver
    combined = friendly_minions + hand_list
    A = [mn.attack for mn in combined]  # attack values
    B = [mn.health for mn in combined]  # health values
    P = [enm.attack for enm in enemy_minions]
    Q = [enm.health for enm in enemy_minions]

    C = [mn.mana_cost for mn in hand_list]
    S = [mn.strat_value for mn in hand_list]

    # 5) Build keyword arrays (Keywords objects) for the solver
    friendly_keywords_list = [mn.keywords for mn in friendly_minions]
    enemy_keywords_list = [emn.keywords for emn in enemy_minions]

    # Optional debug: check how many enemy minions have Taunt or DS
    has_taunt_list = [
        j for j in range(n)
        if enemy_keywords_list[j].has_keyword("Taunt")
    ]
    has_ds_list = [
        j for j in range(n)
        if enemy_keywords_list[j].has_keyword("Divine Shield")
    ]
    print(f"Enemy minions with Taunt: {has_taunt_list}")
    print(f"Enemy minions with Divine Shield: {has_ds_list}")

    # 6) Define or load your weights
    weights = {
        "W1": 12,  # reward for killing hero => (1 - z_hero)
        "W2": 10,  # board clear
        "W3": 5,   # face damage
        "W4": 10,  # penalty for leaving enemy minions alive
        "W5": 2,   # friendly minion survival
        "W6": 12,  # reward for minion->minion damage
        "W7": 2,   # mild penalty if your minions take damage
        "W8": 4    # card-play reward
    }

    # 7) Solve (call your advanced run_single_turn with Taunt & DS constraints)
    solution = run_single_turn(
        m=m, n=n, h=h,
        M=M, H_hero=H_hero,
        A=A, B=B, P=P, Q=Q, C=C, S=S,
        weights=weights,
        friendly_keywords=friendly_keywords_list,
        enemy_keywords=enemy_keywords_list
    )

    # 8) Apply results to scenario_data
    scenario_data = {
        "A": A,
        "B": B,
        "P": P,
        "Q": Q,
        "H_hero": H_hero,
    }
    apply_results(solution, scenario_data)

    # 9) End turn
    end_turn(hero1, hero2)
    print("Turn complete. Inspect logs above for solver outcome.")


if __name__ == "__main__":
    main()

Deck has 4 cards at the start.
Paladin draws: Minion: Bloodfen Raptor | Class: Neutral | Attack: 3 | Health: 2 | Keywords: 
After draw, hand_list has 1 cards.
  Hand card 0: Bloodfen Raptor (Atk=3, HP=2, Cost=2)
Number of friendly minions on board: 4
  - Friendly Minion 0: FriendlyMinion A (Atk=2, HP=2, Keywords=)
  - Friendly Minion 1: FriendlyMinion B (Atk=2, HP=3, Keywords=)
  - Friendly Minion 2: FriendlyMinion C (Atk=3, HP=2, Keywords=)
  - Friendly Minion 3: FriendlyMinion D (Atk=1, HP=1, Keywords=)
Number of enemy minions on board: 3
  - Enemy Minion 0: EnemyMinion #1 (Atk=2, HP=2, Keywords=)
  - Enemy Minion 1: EnemyMinion #2 (Atk=3, HP=3, Keywords=Divine Shield, Taunt)
  - Enemy Minion 2: EnemyMinion #3 (Atk=1, HP=1, Keywords=)
Number of cards in hand: 1
  - Hand Card 0: Bloodfen Raptor (Atk=3, HP=2, Cost=2, Keywords=)
Enemy minions with Taunt: [1]
Enemy minions with Divine Shield: [1]
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model


Optimal solution found (tolerance 1.00e-04)
Best objective 1.100000000000e+02, best bound 1.100000000000e+02, gap 0.0000%
Objective: 110.0
Friendly minion 0 attacked the enemy hero for 2 damage.
Friendly minion 1 attacked enemy minion 0 for 2 damage.
Friendly minion 2 attacked enemy minion 1 for 3 damage.
Friendly minion 3 attacked enemy minion 2 for 1 damage.
Enemy hero health is now 14.
Enemy minion 0 health: 0
Enemy minion 0 has died (health <= 0).
Enemy minion 1 health: 0
Enemy minion 1 has died (health <= 0).
Enemy minion 2 health: 0
Enemy minion 2 has died (health <= 0).

Friendly minions after combat:
  Friendly minion 0 survived with Attack=2, Health=2.
  Friendly minion 1 survived with Attack=2, Health=3.
  Friendly minion 2 did NOT survive (y_survive[2] = 0).
  Friendly minion 3 did NOT survive (y_survive[3] = 0).
  Friendly minion 4 survived with Attack=3, Health=2.

Enemy minions after combat:
  Enemy minion 0 is considered dead (z_enemy[0] = 0).
  Enemy minion 1 is consid